In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import table

from astroquery.mast import Catalogs
from tqdm import tqdm

output_notebook()

Loading BokehJS ...

In [2]:
ttargets = pd.read_csv('../data/GI_S002.csv') # tess 2-min targets
ktargets = pd.read_csv('../data/K2Campaign19targets.csv') # all kepler target

kttargets = pd.read_csv('../data/wtv-K2Campaign19targets.csv') # kepler targets within TESS fov

In [3]:
c = SkyCoord(ra=ttargets.RA.values*u.degree, dec=ttargets.Dec.values*u.degree)  
catalog = SkyCoord(ra=ktargets.rafloat.values*u.degree, dec=ktargets.decfloat.values*u.degree)  
idx, d2d, d3d = c.match_to_catalog_sky(catalog)  

/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:646: RuntimeWarning: invalid value encountered in less
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:646: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:529: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:529: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):


In [4]:
# match things closer than 2 arcseconds
match = d2d.arcsec < 200

In [5]:
d2d.arcsecond.min()

523.8194642865161

In [6]:
# p = figure()
# p.scatter(ttargets.RA, ttargets.Dec, fill_color='black', size=10)
# p.scatter(ktargets.rafloat, ktargets.decfloat, fill_color='red')
# show(p)

# conclusion: no 2-min TESS targets are in the Kepler target list

# ok now lets do the Kepler targets that are in the TESS FFI images

In [7]:
c = SkyCoord(ra=kttargets.RA.values*u.degree, dec=kttargets.DEC.values*u.degree)  
catalog = SkyCoord(ra=ktargets.rafloat.values*u.degree, dec=ktargets.decfloat.values*u.degree)  
idx, d2d, d3d = c.match_to_catalog_sky(catalog)  

/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:646: RuntimeWarning: invalid value encountered in less
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:646: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:529: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/Users/tom/anaconda3/lib/python3.6/site-packages/astropy/coordinates/angles.py:529: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):


In [8]:
matched = ktargets.iloc[idx]

In [9]:
sc = ['SC' in matched.loc[:, 'InvestigationIDs'].values[i] for i in range(matched.shape[0])]

matched[sc]

,EPIC,rafloat,decfloat,Kp,InvestigationIDs


# conclusion: no short cadence targets in TESS FFIs

In [11]:
coords = ['{:f} {:+f}'.format(matched.iloc[i].rafloat, matched.iloc[i].decfloat) for i in range(matched.shape[0])]

jtable = Catalogs.query_region(coords[0], catalog="Tic", radius=0.006)
for c in tqdm(coords[1:]):
    try:
        catalogData = Catalogs.query_region(c, catalog="Tic", radius=0.006)
        jtable = table.vstack([jtable, catalogData[0]])
    except IndexError:
        continue

    

# catalogData[['ID', 'ra', 'dec', 'pmRA', 'pmDEC',
#                        'eclong', 'eclat', 'Tmag', 'Vmag', 'Kmag', 'Teff',
#                        'rad', 'mass', 'd', ]]

100%|██████████| 170/170 [03:47<00:00,  1.34s/it]


In [23]:
jtable.sort(keys='Tmag')
jtable[['ID', 'ra', 'dec', 'pmRA', 'pmDEC',
                       'eclong', 'eclat', 'Tmag', 'Vmag', 'Kmag', 'Teff',
                       'rad', 'mass', 'd', ]]

ID,ra,dec,pmRA,pmDEC,eclong,eclat,Tmag,Vmag,Kmag,Teff,rad,mass,d
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
4554210,347.998285,-11.933584,20.1,-29.9,344.305032,-6.245471,6.676,7.00634,4.917,5136.0,0.895408,0.866131,180.505417
4579916,348.407583,-11.74935,5.04165,-6.87864,344.7484,-6.232866,8.097,9.591,6.124,4034.0,0.650573,0.6307484,439.748138
4582469,348.516774,-11.629373,-21.0234,-2.30602,344.894657,-6.164411,9.411,9.812,8.67,6162.95,1.4575,1.30728,159.107224
4520668,347.867887,-11.852829,7.85538,-9.00657,344.2187,-6.121229,10.513,10.837,9.919,6736.6,1.83542,1.50612,389.4081
404244638,348.158999,-11.899282,--,--,344.464108,-6.275476,10.516,11.396,8.894,4654.0,--,--,--
4579787,348.27578,-11.566559,-25.8715,-52.6024,344.700986,-6.014113,10.674,11.608,9.751,5390.0,1.25636,0.935602,224.76358
4619629,349.260652,-11.217875,-7.95424,-26.0995,345.731571,-6.072171,10.939,11.486,10.019,5835.3,2.4904,1.36365592,258.79
5738944,348.021353,-11.982817,-24.3069,-15.5453,344.306536,-6.299608,11.426,11.982,10.45,5831.0,1.15402,1.05066192,310.613
4582457,348.537033,-11.541905,-16.2565,-1.71591,344.947511,-6.091727,11.575,12.382,10.436,5275.0,0.943261,0.9044225,219.781


In [24]:
jp = jtable.to_pandas()

In [57]:
jp.loc[:,['ID', 'ra', 'dec', 'Tmag', 'Vmag', 'Jmag', 'Teff']]

,ID,ra,dec,Tmag,Vmag,Jmag,Teff
0,4554210,347.998285,-11.933584,6.676,7.00634,7.061,5136.00
1,4579916,348.407583,-11.749350,8.097,9.59100,6.986,4034.00
2,4582469,348.516774,-11.629373,9.411,9.81200,8.949,6162.95
3,4520668,347.867887,-11.852829,10.513,10.83700,10.126,6736.60
4,404244638,348.158999,-11.899282,10.516,11.39600,9.606,4654.00
5,4579787,348.275780,-11.566559,10.674,11.60800,10.162,5390.00
6,4619629,349.260652,-11.217875,10.939,11.48600,10.380,5835.30
7,5738944,348.021353,-11.982817,11.426,11.98200,10.797,5831.00
8,4582457,348.537033,-11.541905,11.575,12.38200,10.939,5275.00
9,404244634,348.109069,-11.962240,11.601,12.80900,10.660,4250.00
